# CHAPTER 8 Data Wrangling: Join, Combine, and Reshape

___This is probably the most intriguing chapter for me, as I've long been confused by the following content...____

- Creat a hierarchical index: passing hierarchical list when creatring a Series or a DataFrame is ok! Quite ease, Hm?
- swap index: `pd.swaplevel('key1','key2')`
- calculate statistics by Level: `frame.sum(axis=1,level='color')`
- turn several columns to the index: `frame.set_index(['c','d'],drop=True/False)`. In contrast, use `frame.reset_index()` to turn index to columns.
- combine:
    - *database combine*: `pd.merge(df1,df2,left_on='key1',right_on='key2',how='outer/inner/left/right/output')`. To merge with multiple keys, pass a list of column names, `pd.merge(df1,df2,on=['key1','key2'])`
    - __Don't use `pd.join()` as it's a special case of `pd.merge()`__
    - for DataFrame, use `pd.concat([df1,df2],axis=1,keys=['level1','level2'],ignore_index=True/False)` to concatenate
    - want to combine data with overlap, use `df1.combine_first(df2)`
- reshaping with hierarchical indexing: `stack()` pivots from columns to the rows; `unstack()` pivots from rows to the columns


In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(
    np.random.randn(9),
    index = [list('aaabbccdd'),[1,2,3,1,3,1,2,2,3]]
    )
data

a  1   -0.097820
   2   -0.259139
   3    0.255741
b  1   -1.945296
   3    0.749121
c  1    0.935677
   2   -0.774922
d  2   -0.106259
   3   -1.257282
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data['b':'c']

b  1   -1.945296
   3    0.749121
c  1    0.935677
   2   -0.774922
dtype: float64

In [5]:
data.loc[['b','d']]

b  1   -1.945296
   3    0.749121
d  2   -0.106259
   3   -1.257282
dtype: float64

In [6]:
data.loc[:,2]

a   -0.259139
c   -0.774922
d   -0.106259
dtype: float64

In [7]:
data.unstack()

,1,2,3
a,-0.097820,-0.259139,0.255741
b,-1.945296,NaN,0.749121
c,0.935677,-0.774922,NaN
d,NaN,-0.106259,-1.257282


In [8]:
data.unstack().stack()

a  1   -0.097820
   2   -0.259139
   3    0.255741
b  1   -1.945296
   3    0.749121
c  1    0.935677
   2   -0.774922
d  2   -0.106259
   3   -1.257282
dtype: float64

In [9]:
frame = pd.DataFrame(
    np.arange(12).reshape((4,3)),
    index = [list('aabb'),[1,2,1,2]],
    columns = [['Ohio','Ohio','Colorado'],['Green','Red','Green']]
)
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [10]:
frame.index.names=['key1','key2']
frame.columns.names=['state','color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [11]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [12]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [13]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [18]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [20]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [21]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [22]:
frame.sum(level='key2')

/var/folders/7l/j9np0sbx1b33249mt44v25980000gn/T/ipykernel_4385/2004046222.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [23]:
frame.sum(axis=1,level='color')

/var/folders/7l/j9np0sbx1b33249mt44v25980000gn/T/ipykernel_4385/3909534327.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(axis=1,level='color')


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [24]:
frame = pd.DataFrame({
    'a':range(7),
    'b':range(7,0,-1),
    'c':['one','one','one','two','two','two','two'],
    'd':[0,1,2,0,1,2,3]
})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [31]:
frame2 = frame.set_index(['c','d'],drop=True)
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [32]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [34]:
df1 = pd.DataFrame({
    'key':list('bbacaab'),
    'data1':range(7)
})
df2 = pd.DataFrame({
    'key':list('abd'),
    'data2':range(3)
})

In [36]:
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


In [38]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [39]:
df3 = pd.DataFrame({
    'lkey':list('bbacaab'),
    'data1':range(7)
})
df4 = pd.DataFrame({
    'rkey':list('abd'),
    'data2':range(3)
})

In [40]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [41]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [43]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey',how='outer')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [44]:
left = pd.DataFrame({
    'key1':['foo','foo','bar'],
    'key2':['one','two','one'],
    'lval':[1,2,3]
})
right = pd.DataFrame({
    'key1':['foo','foo','bar','bar'],
    'key2':['one','one','one','two'],
    'rval':[4,5,6,7]
})


In [45]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [46]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [49]:
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [59]:
lefth = pd.DataFrame({
    'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
    'key2':[2000,2001,2002,2001,2002],
    'data':np.arange(5.)
})
righth= pd.DataFrame(np.arange(12).reshape((6,2)),
                      index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],[2001,2000,2000,2000,2001,2002]],
                      columns = ['event1','event2'])

In [60]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [61]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [64]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [65]:
arr = np.arange(12).reshape((3,4))
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [66]:
s1 = pd.Series([0,1],index=['a','b'])
s2 = pd.Series([2,3,4],index=['c','d','e'])
s3 = pd.Series([5,6],index=['f','g'])

In [71]:
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [72]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [75]:
pd.concat([s1,s4],axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [76]:
result = pd.concat([s1,s2,s3],keys=['one','two','three'])

In [78]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [79]:
df1 = pd.DataFrame(np.arange(6).reshape((3,2)),index=['a','b','c'],columns=['one','two'])
df2 = pd.DataFrame(np.arange(4).reshape((2,2))+5,index=['a','c'],columns=['three','four'])

In [80]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [81]:
df2

,three,four
a,5,6
c,7,8


In [84]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [85]:
a = pd.Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],index=['f','e','d','c','b','a'])
b = pd.Series(np.arange(len(a),dtype=np.float64),index=['f','e','e','c','b','a'])

In [86]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [87]:
b

f    0.0
e    1.0
e    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [88]:
b[-1]=np.nan

In [89]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [90]:
b

f    0.0
e    1.0
e    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [91]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [97]:
df1 = pd.DataFrame({
    'a':[1.,np.nan,5.,np.nan],
    'b':[np.nan,2.,np.nan,6.],
    'c':range(2,18,4)
})
df2 = pd.DataFrame({
    'a':[5,4,np.nan,3,7],
    'b':[np.nan,3,4,6,8]
})

In [98]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [99]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [100]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [101]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index=pd.Index(['Ohio','Colorado'],name='state'),
                   columns = pd.Index(['one','two','three'],name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [102]:
data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [120]:
data = pd.read_csv('macroata.csv')

In [121]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [122]:
periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [123]:
columns = pd.Index(['realgdp','infl','unemp'],name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [124]:
data.reindex(columns=columns)

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [125]:
data.index=periods.to_timestamp('D','end')

In [126]:
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
date,,,,,,,,,,,,,,
1959-03-31 23:59:59.999999999,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1959-06-30 23:59:59.999999999,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
1959-09-30 23:59:59.999999999,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
1959-12-31 23:59:59.999999999,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
1960-03-31 23:59:59.999999999,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-09-30 23:59:59.999999999,2008,3,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
2008-12-31 23:59:59.999999999,2008,4,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
2009-03-31 23:59:59.999999999,2009,1,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71


In [142]:
ldata=data.stack().reset_index().rename(columns={0:'value','level_1':'item'})

In [143]:
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,year,1959.000
1,1959-03-31 23:59:59.999999999,quarter,1.000
2,1959-03-31 23:59:59.999999999,realgdp,2710.349
3,1959-03-31 23:59:59.999999999,realcons,1707.400
4,1959-03-31 23:59:59.999999999,realinv,286.898
...,...,...,...
2837,2009-09-30 23:59:59.999999999,tbilrate,0.120
2838,2009-09-30 23:59:59.999999999,unemp,9.600
2839,2009-09-30 23:59:59.999999999,pop,308.013
2840,2009-09-30 23:59:59.999999999,infl,3.560


In [144]:
pivoted = ldata.pivot('date','item','value')

In [145]:
pivoted

item,cpi,infl,m1,pop,quarter,realcons,realdpi,realgdp,realgovt,realint,realinv,tbilrate,unemp,year
date,,,,,,,,,,,,,,
1959-03-31 23:59:59.999999999,28.980,0.00,139.7,177.146,1.0,1707.4,1886.9,2710.349,470.045,0.00,286.898,2.82,5.8,1959.0
1959-06-30 23:59:59.999999999,29.150,2.34,141.7,177.830,2.0,1733.7,1919.7,2778.801,481.301,0.74,310.859,3.08,5.1,1959.0
1959-09-30 23:59:59.999999999,29.350,2.74,140.5,178.657,3.0,1751.8,1916.4,2775.488,491.260,1.09,289.226,3.82,5.3,1959.0
1959-12-31 23:59:59.999999999,29.370,0.27,140.0,179.386,4.0,1753.7,1931.3,2785.204,484.052,4.06,299.356,4.33,5.6,1959.0
1960-03-31 23:59:59.999999999,29.540,2.31,139.6,180.007,1.0,1770.5,1955.5,2847.699,462.199,1.19,331.722,3.50,5.2,1960.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-09-30 23:59:59.999999999,216.889,-3.16,1474.7,305.270,3.0,9267.7,9838.3,13324.600,991.551,4.33,1990.693,1.17,6.0,2008.0
2008-12-31 23:59:59.999999999,212.174,-8.79,1576.5,305.952,4.0,9195.3,9920.4,13141.920,1007.273,8.91,1857.661,0.12,6.9,2008.0
2009-03-31 23:59:59.999999999,212.671,0.94,1592.8,306.547,1.0,9209.2,9926.4,12925.410,996.287,-0.71,1558.494,0.22,8.1,2009.0


In [146]:
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,year,1959.000
1,1959-03-31 23:59:59.999999999,quarter,1.000
2,1959-03-31 23:59:59.999999999,realgdp,2710.349
3,1959-03-31 23:59:59.999999999,realcons,1707.400
4,1959-03-31 23:59:59.999999999,realinv,286.898
...,...,...,...
2837,2009-09-30 23:59:59.999999999,tbilrate,0.120
2838,2009-09-30 23:59:59.999999999,unemp,9.600
2839,2009-09-30 23:59:59.999999999,pop,308.013
2840,2009-09-30 23:59:59.999999999,infl,3.560


In [148]:
ldata.set_index(['date','item']).unstack('item')

value                                 \
item                               cpi  infl      m1      pop quarter   
date                                                                    
1959-03-31 23:59:59.999999999   28.980  0.00   139.7  177.146     1.0   
1959-06-30 23:59:59.999999999   29.150  2.34   141.7  177.830     2.0   
1959-09-30 23:59:59.999999999   29.350  2.74   140.5  178.657     3.0   
1959-12-31 23:59:59.999999999   29.370  0.27   140.0  179.386     4.0   
1960-03-31 23:59:59.999999999   29.540  2.31   139.6  180.007     1.0   
...                                ...   ...     ...      ...     ...   
2008-09-30 23:59:59.999999999  216.889 -3.16  1474.7  305.270     3.0   
2008-12-31 23:59:59.999999999  212.174 -8.79  1576.5  305.952     4.0   
2009-03-31 23:59:59.999999999  212.671  0.94  1592.8  306.547     1.0   
2009-06-30 23:59:59.999999999  214.469  3.37  1653.6  307.226     2.0   
2009-09-30 23:59:59.999999999  216.385  3.56  1673.9  308.013     3.0   

                                                                              \
item                          realcons  realdpi    realgdp  realgovt realint   
date                                                                           
1959-03-31 23:59:59.999999999   1707.4   1886.9   2710.349   470.045    0.00   
1959-06-30 23:59:59.999999999   1733.7   1919.7   2778.801   481.301    0.74   
1959-09-30 23:59:59.999999999   1751.8   1916.4   2775.488   491.260    1.09   
1959-12-31 23:59:59.999999999   1753.7   1931.3   2785.204   484.052    4.06   
1960-03-31 23:59:59.999999999   1770.5   1955.5   2847.699   462.199    1.19   
...                                ...      ...        ...       ...     ...   
2008-09-30 23:59:59.999999999   9267.7   9838.3  13324.600   991.551    4.33   
2008-12-31 23:59:59.999999999   9195.3   9920.4  13141.920  1007.273    8.91   
2009-03-31 23:59:59.999999999   9209.2   9926.4  12925.410   996.287   -0.71   
2009-06-30 23:59:59.999999999   9189.0  10077.5  12901.504  1023.528   -3.19   
2009-09-30 23:59:59.999999999   9256.0  10040.6  12990.341  1044.088   -3.44   

                                                                
item                            realinv tbilrate unemp    year  
date                                                            
1959-03-31 23:59:59.999999999   286.898     2.82   5.8  1959.0  
1959-06-30 23:59:59.999999999   310.859     3.08   5.1  1959.0  
1959-09-30 23:59:59.999999999   289.226     3.82   5.3  1959.0  
1959-12-31 23:59:59.999999999   299.356     4.33   5.6  1959.0  
1960-03-31 23:59:59.999999999   331.722     3.50   5.2  1960.0  
...                                 ...      ...   ...     ...  
2008-09-30 23:59:59.999999999  1990.693     1.17   6.0  2008.0  
2008-12-31 23:59:59.999999999  1857.661     0.12   6.9  2008.0  
2009-03-31 23:59:59.999999999  1558.494     0.22   8.1  2009.0  
2009-06-30 23:59:59.999999999  1456.678     0.18   9.2  2009.0  
2009-09-30 23:59:59.999999999  1486.398     0.12   9.6  2009.0  

[203 rows x 14 columns]